In [1]:
import requests

In [3]:
OLLAMA_URL = "http://localhost:11434/api/chat"
MODEL = "llama3"

In [40]:
conversation = [
    {
        "role": "system",
        "content": (
            "You are a friendly exam preparation tutor.\n\n"
            "Behavior rules:\n"
            "1) If the student asks a doubt or concept explanation, respond naturally and clearly.\n"
            "2) If the student explicitly requests a format like SHORT ANSWER, LONG ANSWER, or MCQ, "
            "then strictly follow that format.\n"
            "3) Do NOT force formats if the student did not ask for them.\n"
            "4) Use simple language suitable for students.\n"
            "5) Be accurate and exam-oriented when answering tests.\n"
            "6) If the student asks for a test or practice questions but does NOT specify the format, "
            "politely ask them which format they want (SHORT ANSWER, LONG ANSWER, or MCQ) before answering."
            "7) If the student submits an answer for evaluation, evaluate it and provide constructive feedback."
        )
    }
]


In [21]:
def ask_exam_bot(question):
    conversation.append({"role": "user", "content": question})

    payload = {
        "model": MODEL,
        "messages": conversation,
        "stream": False
    }

    response = requests.post(OLLAMA_URL, json=payload)
    response.raise_for_status()

    reply = response.json()["message"]["content"]
    conversation.append({"role": "assistant", "content": reply})

    return reply

In [26]:
def generate_model_answer(question, marks=5):
    prompt = (
        f"Generate an ideal exam answer for the following question.\n"
        f"Question: {question}\n"
        f"Marks: {marks}\n\n"
        f"Requirements:\n"
        f"- Exam-oriented\n"
        f"- Covers all key points for full marks\n"
        f"- Clear and structured"
    )
    
    return ask_exam_bot(prompt)


In [28]:
def evaluate_answer_with_difficulty(question, student_answer, difficulty="Medium", marks=5):
    prompt = (
        f"QUESTION:\n{question}\n\n"
        f"STUDENT ANSWER:\n{student_answer}\n\n"
        f"Evaluate the answer for {marks} marks.\n"
        f"Difficulty level: {difficulty}\n\n"
        f"Evaluation rules:\n"
        f"- Easy: Be lenient, focus on basic understanding\n"
        f"- Medium: Balance understanding and structure\n"
        f"- Hard: Be strict, check completeness and accuracy\n\n"
        f"Provide:\n"
        f"1) Score out of {marks}\n"
        f"2) Strengths\n"
        f"3) Weaknesses / Missing points\n"
        f"4) How to improve"
    )
    
    return ask_exam_bot(prompt)


In [30]:
student_progress = []

def save_attempt(question, student_answer, evaluation, difficulty, marks):
    student_progress.append({
        "question": question,
        "answer": student_answer,
        "evaluation": evaluation,
        "difficulty": difficulty,
        "marks": marks
    })


In [32]:
def exam_practice(question, student_answer, marks=5, difficulty="Medium"):
    print("📝 Question:")
    print(question)
    
    print("\n📘 Model Answer:")
    model_answer = generate_model_answer(question, marks)
    print(model_answer)
    
    print("\n📊 Evaluation:")
    evaluation = evaluate_answer_with_difficulty(
        question, student_answer, difficulty, marks
    )
    print(evaluation)
    
    save_attempt(question, student_answer, evaluation, difficulty, marks)
    
    print("\n✅ Attempt saved.")


In [34]:
for i, attempt in enumerate(student_progress, 1):
    print(f"\nAttempt {i}")
    print("Question:", attempt["question"])
    print("Difficulty:", attempt["difficulty"])
    print("Evaluation:", attempt["evaluation"])


In [17]:
def reset_exam():
    global conversation
    conversation = conversation[:1]  # keep system prompt only


In [36]:
print(ask_exam_bot("What does CPU stand for?"))

CPU stands for Central Processing Unit.


In [38]:
exam_practice("What are iput devices?","Keyboard, mouse, joystick are some examples of input devices")

📝 Question:
What are iput devices?

📘 Model Answer:
Here's a sample ideal exam answer:

Input devices are electronic tools that allow users to provide data or instructions to a computer. They enable humans to interact with the digital world, feeding information into the system for processing, storage, or display. Key examples of input devices include:

* Keyboard: used for typing commands, writing documents, and communicating through email or chat.
* Mouse: employed for pointing and clicking on graphical user interfaces (GUIs), selecting options, and navigating menus.
* Scanner: capable of capturing images from physical sources such as paper documents, photos, or objects, converting them into digital formats.
* Webcam: captures live video feed, allowing users to engage in video conferencing, take selfies, or record video messages.
* Microphone: converts spoken words or sounds into digital signals, facilitating voice-to-text functionality, online meetings, or voice commands.

In summary